In [ ]:
#default_exp data.pipeline

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.transform import *
from local.notebook.showdoc import show_doc

In [ ]:
#hide
torch.cuda.set_device(int(os.environ.get('DEFAULT_GPU') or 0))

# Pipeline

> Low-level transform pipelines

The classes here provide functionality for creating *partially reversible functions*, which we call `Transform`s. By "partially reversible" we mean that a transform can be `decode`d, creating a form suitable for display. This is not necessarily identical to the original form (e.g. a transform that changes a byte tensor to a float tensor does not recreate a byte tensor when decoded, since that may lose precision, and a float tensor can be displayed already.)

Classes are also provided and for composing transforms, and mapping them over collections. The following functionality is provided:

- A `Transform` is created with an `encodes` and potentially `decodes` function. 
- `Pipeline` is a transform which composes transforms
- `TfmdList` takes a collection and a transform, and provides an indexer (`__getitem__`) which dynamically applies the transform to the collection items.
- `Tuplify` is a special `Trannsform` that takes a list of list of transforms or a list of `Pipeline`s, then aapplies them to the element it receives to return a tuple.

## Convenience functions

In [ ]:
#export
def get_func(t, name, *args, **kwargs):
    "Get the `t.name` (potentially partial-ized with `args` and `kwargs`) or `noop` if not defined"
    f = getattr(t, name, noop)
    return f if not (args or kwargs) else partial(f, *args, **kwargs)

This works for any kind of `t` supporting `getattr`, so a class or a module.

In [ ]:
test_eq(get_func(operator, 'neg', 2)(), -2)
test_eq(get_func(operator.neg, '__call__')(2), -2)
test_eq(get_func(list, 'foobar')([2]), [2])
t = get_func(torch, 'zeros', dtype=torch.int64)(5)
test_eq(t.dtype, torch.int64)
a = [2,1]
get_func(list, 'sort')(a)
test_eq(a, [1,2])

## Func -

Tranforms, are built with multiple-dispatch: a given function can have several methods depending on the type of the object received. This is done directly with the `multimethod` module and type-annotation in `Transofrm`, but you can also use the following class.

In [ ]:
#export
class Func():
    "Basic wrapper around a `name` with `args` and `kwargs` to call on a given type"
    def __init__(self, name, *args, **kwargs): self.name,self.args,self.kwargs = name,args,kwargs
    def __repr__(self): return f'sig: {self.name}({self.args}, {self.kwargs})'
    def _get(self, t): return get_func(t, self.name, *self.args, **self.kwargs)
    def __call__(self,t): return L(t).mapped(self._get) if is_listy(t) else self._get(t)

You can call the `Func` object on any module name or type, even a list of types. It will return the corresponding function (with a default to `noop` if nothing is found) or list of functions.

In [ ]:
test_eq(Func('sqrt')(math), math.sqrt)
test_eq(Func('sqrt')(torch), torch.sqrt)

@patch
def powx(x:math, a): return math.pow(x,a)
@patch
def powx(x:torch, a): return torch.pow(x,a)
tst = Func('powx',a=2)([math, torch])
test_eq([f.func for f in tst], [math.powx, torch.powx])
for t in tst: test_eq(t.keywords, {'a': 2})

In [ ]:
#export
class _Sig():
    def __getattr__(self,k):
        def _inner(*args, **kwargs): return Func(k, *args, **kwargs)
        return _inner

Sig = _Sig()

In [ ]:
show_doc(Sig, name="Sig")

<h4 id="<code>Sig</code>" class="doc_header"><code>Sig</code><a href="https://github.com/fastai/fastai_docs/tree/master/dev/__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>Sig</code>(**\*`args`**, **\*\*`kwargs`**)



`Sig` is just sugar-syntax to create a `Func` object more easily with the syntax `Sig.name(*args, **kwargs)`.

In [ ]:
f = Sig.sqrt()
test_eq(f(math), math.sqrt)
test_eq(f(torch), torch.sqrt)

In [ ]:
#export
class SelfFunc():
    "Search for `name` attribute and call it with `args` and `kwargs` on any object it's passed."
    def __init__(self, nm, *args, **kwargs): self.nm,self.args,self.kwargs = nm,args,kwargs
    def __repr__(self): return f'self: {self.nm}({self.args}, {self.kwargs})'
    def __call__(self, o):
        if not is_listy(o): return getattr(o,self.nm)(*self.args, **self.kwargs)
        else: return [getattr(o_,self.nm)(*self.args, **self.kwargs) for o_ in o]

The difference between `Func` and `SelfFunc` is that `Func` will generate a function when you call it on a type. On the other hand, `SelfFunc` is already a function and each time you call it on an object it looks for the `name` attribute and call it on `args` and `kwargs`.

In [ ]:
tst = SelfFunc('sqrt')
x = torch.tensor([4.])
test_eq(tst(x), torch.tensor([2.]))
assert isinstance(tst(x), Tensor)

In [ ]:
#export
class _SelfFunc():
    def __getattr__(self,k):
        def _inner(*args, **kwargs): return SelfFunc(k, *args, **kwargs)
        return _inner
    
Self = _SelfFunc()

In [ ]:
show_doc(Self, name="Self")

<h4 id="<code>Self</code>" class="doc_header"><code>Self</code><a href="https://github.com/fastai/fastai_docs/tree/master/dev/__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>Self</code>(**\*`args`**, **\*\*`kwargs`**)



`Self` is just syntax sugar to create a `SelfFunc` object more easily with the syntax `Self.name(*args, **kwargs)`.

In [ ]:
f = Self.sqrt()
x = torch.tensor([4.])
test_eq(f(x), torch.tensor([2.]))
assert isinstance(f(x), Tensor)

## Pipeline -

In [ ]:
class Int(int):
    def show(self, ctx=None, **kwargs): return show_title(str(self), ctx=ctx)
class Float(float):
    def show(self, ctx=None, **kwargs): return show_title(str(self), ctx=ctx)
class Str(str):
    def show(self, ctx=None, **kwargs): return show_title(self, ctx=ctx)

def to_int  (x)->Int  : return x
def to_float(x)->Float: return x
def to_none (x)->None : return x
def double(x): return x*2
def half  (x): return x/2

In [ ]:
#export
def compose_tfms(x, tfms, func_nm=None, reverse=False, **kwargs):
    "Apply all `func_nm` attribute of `tfms` on `x`, maybe in `reverse` order"
    if reverse: tfms = reversed(tfms)
    r = NoneType
    for f in tfms:
        if func_nm: f = getattr(f,func_nm,noop)
        r = anno_ret(f) or r
        x = f(x, **kwargs)
        if r!=NoneType: x=r(x)
    return x

In [ ]:
# TODO: compose all tfm types with return anno (do casting and ret type inside tfm?)

In [ ]:
test_eq_type(compose_tfms(1, [to_int]), Int(1))
test_eq_type(compose_tfms(1, [to_int,to_float]), Float(1))
test_eq_type(compose_tfms(1, [to_int,to_float,double]), Float(2))
test_eq_type(compose_tfms(2.0, [to_int,double,half]), Int(2))
test_eq_type(compose_tfms(2.0, [to_int,to_none,double,half]), 2.0)

In [ ]:
class _AddOne(TupleTransform):
    def encodes(self, x:float): return x+1
    def decodes(self, x): return x-1
    
tfms = [_AddOne(), TupleTransform(math.sqrt)]
t = compose_tfms(3., tfms)
test_eq(t, 2.)
test_eq(compose_tfms(t, tfms, 'decodes'), 1.)
test_eq(compose_tfms(4., tfms, reverse=True), 3.)
test_eq(compose_tfms((9,3.), tfms), (3,2.))

In [ ]:
#export
class Pipeline():
    "A pipeline of composed (for encode/decode) transforms, setup with types"
    def __init__(self, funcs=None): 
        if isinstance(funcs, Pipeline): funcs = funcs.fs
        self.fs = []
        for i,f in enumerate(L(funcs).sorted(key='order')):
            if not isinstance(f,BasicTransform): f = TupleTransform(f)
            self.fs.append(f)
    
    def __repr__(self): return f"Pipeline: {self.fs}"
    
    def setup(self, items=None):
        tfms,self.fs = self.fs,[]
        for t in tfms:
            if t.add_before_setup:     self.fs.append(t)
            if hasattr(t, 'setup'):    t.setup(items)
            if not t.add_before_setup: self.fs.append(t)
                
    def __call__(self, o, filt=None): return compose_tfms(o, self.fs, filt=filt)
    def decode  (self, i, filt=None): return compose_tfms(i, self.fs, func_nm='decode', reverse=True, filt=filt)
    
    def show(self, o, ctx=None, filt=None, **kwargs):
        for f in reversed(self.fs):
            if hasattr(o, 'show'): return o.show(ctx)
            o = f.decode(o, filt=filt)
        if hasattr(o, 'show'): return o.show(ctx)

In [ ]:
add_docs(Pipeline,
         __call__="Compose `__call__` of all `tfms` on `o`",
         decode="Compose `decode` of all `tfms` on `i`",
         show="Show item `o`",
         setup="Go through the transforms in order and call their potential setup on `items`")

A list of transforms are often applied in a particular order, and decoded by applying in the reverse order. `Pipeline` provides this functionality, and also ensures during its initialization that each transform get the proper functions according to the type of the previous transform. If any transform provides a type with a return annotation, this type is passed along to the next tranforms (until being overwritten by a new return annotation). Such a type can be useful when transforms filter depending on a given type (usually for data augmentation) or to provide a show method.

Here's some simple examples:

In [ ]:
# Empty pipeline is noop
pipe = Pipeline()
test_eq(pipe(1), 1)

In [ ]:
class IntFloatTfm(TupleTransform):
    def encodes(self, x)->Int: return x
    def decodes(self, x)->Float: return x

In [ ]:
int_tfm=IntFloatTfm()
def neg(x): return -x
neg_tfm = TupleTransform(neg, neg)
    
pipe = Pipeline([neg_tfm, int_tfm])
start = 2.0
t = pipe(start)
test_eq(t, -2)
test_eq(type(t), Int)
test_eq(pipe.decode(t), start)
test_stdout(lambda:pipe.show(t), '-2')

In [ ]:
# Check opposite order
pipe = Pipeline([int_tfm,neg_tfm])
t = pipe(start)
test_eq(t, -2)
test_stdout(lambda:pipe.show(t), '2.0')

In [ ]:
class _AddOne(Transform):
    def encodes(self, x): return x+1
    def decodes(self, x): return x-1

- decode predictions
- handle batches (always tensors)
- pass final type on to dl
- wrap with most recent return anno, to avoid eg aug reverting to plain tensor

In [ ]:
#Check filtering is properly applied
add1 = _AddOne()
add1.filt = 1
pipe = Pipeline([neg_tfm, int_tfm, add1])
test_eq(pipe(start), -2)
test_eq(pipe(start, filt=1), -1)
test_eq(pipe(start, filt=0), -2)
for t in [None, 0, 1]: test_eq(pipe.decode(pipe(start, filt=t), filt=t), start)
for t in [None, 0, 1]: test_stdout(lambda: pipe.show(pipe(start, filt=t), filt=t), "2")

AssertionError: ==:
-2
-1

In [ ]:
#Check type is properly changed at dispatch
class _AddOne(Transform):
    def encodes(self, x:int)->String: return x+1
    def encodes(self, x:float):       return x*2
    def decodes(self, x:int):   return x-1
    def decodes(self, x:float): return x/2

pipe = Pipeline(_AddOne(), t=int)
test_eq(pipe.final_t, String)
pipe = Pipeline(_AddOne(), t=float)
test_eq(pipe.final_t, float)
pipe = Pipeline(_AddOne(), t=[int,float])
test_eq(pipe.final_t, [String,float])

### Methods

In [ ]:
show_doc(Pipeline.__call__)

<h4 id="<code>Pipeline.__call__</code>" class="doc_header"><code>Pipeline.__call__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.__call__</code>(**`o`**, **`filt`**=*`None`*)

Compose `__call__` of all `tfms` on `o`

In [ ]:
show_doc(Pipeline.decode)

<h4 id="<code>Pipeline.decode</code>" class="doc_header"><code>Pipeline.decode</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.decode</code>(**`i`**, **`filt`**=*`None`*)

Compose `decode` of all `tfms` on `i`

In [ ]:
show_doc(Pipeline.new)

<h4 id="<code>Pipeline.new</code>" class="doc_header"><code>Pipeline.new</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.new</code>(**`t`**=*`None`*)

Create a new [`Pipeline`](/data.pipeline.html#Pipeline)with the same `tfms` and a new initial `t`

In [ ]:
show_doc(Pipeline.setup)

<h4 id="<code>Pipeline.setup</code>" class="doc_header"><code>Pipeline.setup</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#Pipeline--" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.setup</code>(**`items`**=*`None`*)

Go through the transforms in order and call their potential setup on `items`

During the setup, the `Pipeline` starts with no transform and adds them one at a time, so that during its setup, each transfrom get the items processed up to its point and not after. Depending on the attribute `add_before_setup`, the transform is added after the setup (default behaivor) so it's not called on the items used for the setup, or before (in which case it's called on the values used for setup).

In [ ]:
#hide
#Test is below with TfmdList

## TfmedList -

In [ ]:
#export
def get_samples(b, max_rows=10):
    if isinstance(b, Tensor): return b[:max_rows]
    return zip(*L(get_samples(b_, max_rows) if not isinstance(b,Tensor) else b_[:max_rows] for b_ in b))

In [ ]:
#export
@docs
class TfmdList(GetAttr):
    "A `Pipeline` of `tfms` applied to a collection of `items`"
    _xtra = 'decode __call__ show'.split()
    
    def __init__(self, items, tfms, do_setup=True, parent=None):
        self.items,self.parent = L(items),parent
        self.default = self.tfms = Pipeline(tfms)
        if do_setup: self.setup()

    def __getitem__(self, i, filt=None):
        "Transformed item(s) at `i`"
        its = self.items[i]
        if is_iter(i):
            if not is_iter(filt): filt = L(filt for _ in i)
            return L(self.tfms(it, filt=f) for it,f in zip(its,filt))
        return self.tfms(its, filt=filt)
    
    def setup(self): self.tfms.setup(self)
    def subset(self, idxs): return self.__class__(self.items[idxs], self.tfms, do_setup=False, parent=self)
    def decode_at(self, idx, filt=None): 
        return self.decode(self.__getitem__(idx,filt=filt), filt=filt)
    def show_at(self, idx, filt=None, **kwargs): 
        return self.show(self.__getitem__(idx,filt=filt), filt=filt, **kwargs)
    def __eq__(self, b): return all_equal(self, b)
    def __len__(self): return len(self.items)
    def __iter__(self): return (self[i] for i in range_of(self))
    def __repr__(self): return f"{self.__class__.__name__}: {self.items}\ntfms - {self.tfms}"
     
    _docs = dict(setup="Transform setup with self",
                 decode_at="Decoded item at `idx`",
                 show_at="Show item at `idx`",
                 subset="New `TfmdList` that only includes items at `idxs`")

`tfms` can either be a `Pipeline` or a list of transforms.

In [ ]:
tl = TfmdList([1,2,3], [neg_tfm, float_tfm])
t = tl[1]
test_eq(t, -2.0)
test_eq(type(t), float)
test_eq(tl.decode_at(1), 2)
test_eq(tl.decode(t), 2)
test_stdout(lambda: tl.show_at(2), '3')
tl

TfmdList: (#3) [1,2,3]
tfms - Pipeline over [<__main__.Transform object at 0x7f6816f55c18>, <__main__.floatTfm object at 0x7f6816f55b38>]

In [ ]:
p2 = tl.subset([0,2])
test_eq(p2, [-1.,-3.])

Here's how we can use `TfmdList.setup` to implement a simple category list, getting labels from a mock file list:

In [ ]:
class _Cat(Transform):
    order = 1
    def __init__(self, subset_idx=None): self.subset_idx = subset_idx
    def encodes(self, o): return self.o2i[o]
    def decodes(self, o): return self.vocab[o]
    def setup(self, items): 
        if self.subset_idx is not None: items = items.subset(self.subset_idx)
        self.vocab,self.o2i = uniqueify(items, sort=True, bidir=True)

def _lbl(o) -> String: return o.split('_')[0]

test_fns = ['dog_0.jpg','cat_0.jpg','cat_2.jpg','cat_1.jpg','dog_1.jpg']
tcat = _Cat()
tl = TfmdList(test_fns, [tcat,_lbl])

test_eq(tcat.vocab, ['cat','dog'])
test_eq([1,0,0,0,1], tl)
test_eq(1, tl[-1])
test_eq([1,0], tl[0,1])
t = list(tl)
test_eq([1,0,0,0,1], t)
test_eq(['dog','cat','cat','cat','dog'], map(tl.decode,t))
test_stdout(lambda:tl.show_at(0), "dog")
tl

TfmdList: (#5) [dog_0.jpg,cat_0.jpg,cat_2.jpg,cat_1.jpg,dog_1.jpg]
tfms - Pipeline over [<__main__.Transform object at 0x7f6816f587f0>, <__main__._Cat object at 0x7f6816f58588>]

In [ ]:
tcat = _Cat([0,1,2])
tl = TfmdList(test_fns, [tcat,_lbl])

In [ ]:
#hide
#Test of add_before_setup
class _AddSome(Transform):
    def __init__(self):   self.a = 2
    def encodes(self, x): return x+self.a
    def decodes(self, x): return x-self.a
    def setup(self, items): self.a = tensor(items).float().mean().item()
        
tl1 = TfmdList([1,2,3,4], _AddSome())
test_eq(tl1.tfms.fs[0].a, 2.5) #Setup on the raw items, mean is 2.5

_AddSome.add_before_setup = True
tl1 = TfmdList([1,2,3,4], _AddSome())
test_eq(tl1.tfms.fs[0].a, 4.5) #Setup on the tfmed items, mean is 4.5

In [ ]:
#hide
#Check filtering is properly applied
tl1 = TfmdList([1,2,3,4], [neg_tfm, float_tfm, _FiltAddOne()])
test_eq(tl1[2], -3)
test_eq(tl1.__getitem__(2, filt=1), -2)
test_eq(tl1.__getitem__(2, filt=0), -3)
test_eq(tl1.__getitem__([2,2], filt=[0,1]), [-3,-2])
for t in [None, 0, 1]: test_eq(tl1.decode(tl1.__getitem__(1, filt=t), filt=t), 2)
for t in [None, 0, 1]: test_eq(tl1.decode_at(1, filt=t), 2)
for t in [None, 0, 1]: test_stdout(lambda: tl1.show_at(1, filt=t), "2")

### Methods

In [ ]:
show_doc(TfmdList.__getitem__)

<h4 id="<code>TfmdList.__getitem__</code>" class="doc_header"><code>TfmdList.__getitem__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmedList--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdList.__getitem__</code>(**`i`**, **`filt`**=*`None`*)

Transformed item(s) at `i`

In [ ]:
show_doc(TfmdList.decode_at)

<h4 id="<code>TfmdList.decode_at</code>" class="doc_header"><code>TfmdList.decode_at</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmedList--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdList.decode_at</code>(**`idx`**, **`filt`**=*`None`*)

Decoded item at `idx`

In [ ]:
test_eq(tl.decode_at(1),tl.decode(tl[1]))

In [ ]:
show_doc(TfmdList.show_at)

<h4 id="<code>TfmdList.show_at</code>" class="doc_header"><code>TfmdList.show_at</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmedList--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdList.show_at</code>(**`idx`**, **`filt`**=*`None`*, **\*\*`kwargs`**)

Show item at `idx`

In [ ]:
test_stdout(lambda: tl.show_at(1), 'cat')

In [ ]:
show_doc(TfmdList.subset)

<h4 id="<code>TfmdList.subset</code>" class="doc_header"><code>TfmdList.subset</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmedList--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdList.subset</code>(**`idxs`**)

New [`TfmdList`](/data.pipeline.html#TfmdList) that only includes items at `idxs`

## TfmdDS -

In [ ]:
#exports
def _maybe_flat(t): return t[0] if len(t) == 1 else tuple(t)

In [ ]:
#export
class TfmdDS(TfmdList):
    def __init__(self, items, type_tfms=None, ds_tfms=None, do_setup=True):
        if type_tfms is None: type_tfms = [None]
        self.items = items
        self.tfmd_its = [TfmdList(items, t, do_setup=do_setup, parent=self) for t in type_tfms]
        self.__post_init__(items, ds_tfms, do_setup)
    
    def __post_init__(self, items, ds_tfms, do_setup):
        #To avoid dupe code with DataSource
        self.type_tfms = [it.tfms for it in self.tfmd_its]
        self.ds_tfms = Pipeline(ds_tfms, t=[t_.final_t for t_ in self.type_tfms])
        if do_setup: self.setup()
        
    def __getitem__(self, i, filt=None):
        its = _maybe_flat([it.__getitem__(i, filt=filt) for it in self.tfmd_its])
        if is_iter(i): 
            if len(self.tfmd_its) > 1: its = zip(*L(its))
            if not is_iter(filt): filt = L(filt for _ in i)
            return L(self.ds_tfms(it, filt=f) for it,f in zip(its,filt))
        return self.ds_tfms(its, filt=filt)
    
    def decode(self, o, filt=None):
        o = self.ds_tfms.decode(o, filt=filt)
        if not is_listy(o): o = [o]
        return _maybe_flat([it.decode(o_, filt=filt) for o_,it in zip(o,self.tfmd_its)])
    
    def decode_batch(self, b, filt=None): return [self.decode(b_, filt=filt) for b_ in get_samples(b)]
    
    def show(self, o, ctx=None, filt=None, **kwargs):
        if self.ds_tfms.t_show is not None: return self.ds_tfms.show(o, ctx=ctx, filt=filt, **kwargs)
        o = self.ds_tfms.decode(o, filt=filt)
        if not is_listy(o): o = [o]
        for o_,it in zip(o,self.tfmd_its): ctx = it.show(o_, ctx=ctx, filt=filt, **kwargs)
        return ctx
    
    def setup(self): self.ds_tfms.setup(self)
        
    def subset(self, idxs): 
        return self.__class__(self.items[idxs], self.type_tfms, self.ds_tfms, do_setup=False)
    
    def __repr__(self): 
        return f"{self.__class__.__name__}: {self.items}\ntype tfms - {self.type_tfms}\nds tfms - {self.ds_tfms}"

In [ ]:
#export 
add_docs(TfmdDS,
         "A `Dataset` created from raw `items` by calling each element of `tfms` on them",
         __getitem__="Call all `tfms` on `items[i]` then all `tuple_tfms` on the result",
         decode="Compose `decode` of all `tuple_tfms` then all `tfms` on `i`",
         decode_batch="`decode` all sample in a the batch `b`",
         show="Show item `o` in `ctx`",
         setup="Go through the transforms in order and call their potential setup on `items`",
         subset="New `TfmdDS` that only includes items at `idxs`")

`tfms` is a list of objects that can be:
- one transform
- a list of transforms
- a `Pipeline`

In [ ]:
#export 
def setattr_parent(o, k, v):
    if getattr(o, 'parent', False): setattr_parent(o.parent, k, v)
    if getattr(o, 'dsrc', False): setattr_parent(o.dsrc, k, v)
    setattr(o, k, v)

In [ ]:
class _TNorm(Transform):
    def encodes(self, o): return (o-self.m)/self.s
    def decodes(self, o): return (o*self.s)+self.m
    def setup(self, items):
        its = tensor(items).float()
        self.m,self.s = its.mean(),its.std()
        setattr_parent(items, 'm', self.m)
        setattr_parent(items, 's', self.s)

In [ ]:
items = [1,2,3,4]
#If you pass only a list with one list of tfms/Pipeline, TfmdDS behaves like TfmOver
tds = TfmdDS(items, [neg_tfm])
test_eq(tds[0], -1)
test_eq(tds[[0,1,2]], [-1,-2,-3])
test_eq(tds.decode_at(0), 1)
test_stdout(lambda:tds.show_at(1), '2')

In [ ]:
items = [1,2,3,4]
tds = TfmdDS(items, [neg_tfm, [neg_tfm,_TNorm()]])
x,y = zip(*tds)
test_close(tensor(y).mean(), 0)
test_close(tensor(y).std(), 1)
test_eq(x, [-1,-2,-3,-4])
test_stdout(lambda:tds.show_at(1), '2\ntensor(2.)')
test_eq(tds.m, tds.type_tfms[1].fs[1].m)
test_eq(tds.s, tds.type_tfms[1].fs[1].s)
#Attributes are set to all parents progressively
test_eq(tds.tfmd_its[1].m, tds.type_tfms[1].fs[1].m)
test_eq(tds.tfmd_its[1].s, tds.type_tfms[1].fs[1].s)

In [ ]:
#hide
#Test if show at the tuple level interrupts decoding
class DoubleString():
    @staticmethod
    def show(o, ctx=None, **kwargs): print(o[0],o[1])

class _DummyTfm(Transform):
    def encodes(self, x,y)->DoubleString: return [x,y]

items = [1,2,3,4]
tds = TfmdDS(items, [neg_tfm, neg_tfm], _DummyTfm())
test_stdout(lambda: tds.show_at(0), "-1 -1")

In [ ]:
#hide
#Check filtering is properly applied
tds = TfmdDS(items, [neg_tfm, [neg_tfm,_FiltAddOne()]])
test_eq(tds[1], [-2,-2])
test_eq(tds.__getitem__(1, filt=1), [-2,-1])
test_eq(tds.__getitem__(1, filt=0), [-2,-2])
test_eq(tds.__getitem__([1,1], filt=[0,1]), [[-2,-2], [-2,-1]])
for t in [None, 0, 1]: test_eq(tds.decode(tds.__getitem__(1, filt=t), filt=t), [2,2])
for t in [None, 0, 1]: test_eq(tds.decode_at(1, filt=t), [2,2])
for t in [None, 0, 1]: test_stdout(lambda: tds.show_at(1, filt=t), "2\n2")

In [ ]:
tds.__getitem__([1,1], filt=[0,1])

(#2) [(#2) [-2,-2],(#2) [-2,-1]]

In [ ]:
show_doc(TfmdDS.__getitem__)

<h4 id="<code>TfmdDS.__getitem__</code>" class="doc_header"><code>TfmdDS.__getitem__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmdDS--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdDS.__getitem__</code>(**`i`**, **`filt`**=*`None`*)

Call all `tfms` on `items[i]` then all `tuple_tfms` on the result

In [ ]:
show_doc(TfmdDS.decode)

<h4 id="<code>TfmdDS.decode</code>" class="doc_header"><code>TfmdDS.decode</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmdDS--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdDS.decode</code>(**`o`**, **`filt`**=*`None`*)

Compose `decode` of all `tuple_tfms` then all `tfms` on `i`

In [ ]:
show_doc(TfmdDS.decode_at)

<h4 id="<code>TfmdList.decode_at</code>" class="doc_header"><code>TfmdList.decode_at</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmedList--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdList.decode_at</code>(**`idx`**, **`filt`**=*`None`*)

Decoded item at `idx`

In [ ]:
show_doc(TfmdDS.show)

<h4 id="<code>TfmdDS.show</code>" class="doc_header"><code>TfmdDS.show</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmdDS--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdDS.show</code>(**`o`**, **`ctx`**=*`None`*, **`filt`**=*`None`*, **\*\*`kwargs`**)

Show item `o` in `ctx`

In [ ]:
show_doc(TfmdDS.show_at)

<h4 id="<code>TfmdList.show_at</code>" class="doc_header"><code>TfmdList.show_at</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmedList--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdList.show_at</code>(**`idx`**, **`filt`**=*`None`*, **\*\*`kwargs`**)

Show item at `idx`

In [ ]:
show_doc(TfmdDS.setup)

<h4 id="<code>TfmdDS.setup</code>" class="doc_header"><code>TfmdDS.setup</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmdDS--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdDS.setup</code>()

Go through the transforms in order and call their potential setup on `items`

In [ ]:
show_doc(TfmdDS.subset)

<h4 id="<code>TfmdDS.subset</code>" class="doc_header"><code>TfmdDS.subset</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/02_data_pipeline.ipynb#TfmdDS--" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdDS.subset</code>(**`idxs`**)

New [`TfmdDS`](/data.pipeline.html#TfmdDS) that only includes items at `idxs`

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 02_data_pipeline.ipynb.
Converted 03_data_external.ipynb.
Converted 04_data_core.ipynb.
Converted 05_data_source.ipynb.
Converted 06_vision_core.ipynb.
Converted 07_pets_tutorial-meta.ipynb.
Converted 07_pets_tutorial.ipynb.
Converted 08_vision_augment.ipynb.
Converted 09_data_block.ipynb.
Converted 10_layers.ipynb.
Converted 11_optimizer.ipynb.
Converted 12_learner.ipynb.
Converted 13_callback_schedule.ipynb.
Converted 14_callback_hook.ipynb.
Converted 15_callback_progress.ipynb.
Converted 16_callback_tracker.ipynb.
Converted 17_callback_fp16.ipynb.
Converted 30_text_core.ipynb.
Converted 90_notebook_core.ipynb.
Converted 91_notebook_export.ipynb.
Converted 92_notebook_showdoc.ipynb.
Converted 93_notebook_export2html.ipynb.
Converted 94_index.ipynb.
Converted 95_synth_learner.ipynb.
